In [ ]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

--2021-12-02 13:49:04--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.78.216.154, 2600:1417:76:49f::e59, 2600:1417:76:495::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.78.216.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M   108MB/s    in 7.5s    

2021-12-02 13:49:12 (105 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]



In [ ]:
!unzip kagglecatsanddogs_3367a.zip

In [ ]:
import shutil
import os

In [ ]:
#Каталог с набором данных
data_dir = '/content/PetImages/'
#Каталог с данными для обучения
train_dir = 'train'
#Каталог с данными для проверки
val_dir   = 'val'
#Каталог с данными для тестирования
test_dir  = 'test'
#Часть набора данных для тестривания
test_portion = 0.15
#Часть набора данных для проверки
val_portion = 0.15
#Количество элементов данных в одном классе
nb_images = 12500

In [ ]:
def create_dir(dir_name):
  
  if os.path.exists(dir_name):
    shutil.rmtree(dir_name)

  os.makedirs(dir_name)
  os.makedirs(os.path.join(dir_name, "cats"))
  os.makedirs(os.path.join(dir_name, "dogs"))

In [ ]:
create_dir(train_dir)
create_dir(test_dir)
create_dir(val_dir)

In [ ]:
def copy_images(st, ed, src, dst):
  for i in range(st, ed):
    shutil.copy2(os.path.join(src, "Cat/" + str(i) + ".jpg"), os.path.join(dst, "cats"))
    shutil.copy2(os.path.join(src, "Dog/" + str(i) + ".jpg"), os.path.join(dst, "dogs"))

In [ ]:
start_val = int(nb_images * (1 - val_portion - test_portion))
start_tst = int(nb_images * (1 - test_portion))
print(start_val)
print(start_tst)
# train - 0..8749
# val   - 8750 .. 10624
# test  - 10625 .. 12500

8750
10625


In [ ]:
# train
copy_images(0, start_val, data_dir, train_dir)

In [ ]:
copy_images(start_val, start_tst, data_dir, val_dir)

In [ ]:
copy_images(start_tst, nb_images, data_dir, test_dir)

In [ ]:
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
epochs = 2
batch_size = 128
nb_train   = 17500
nb_val     = 3750
nb_test    = 3750

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = datagen.flow_from_directory(train_dir, 
                                              target_size=(img_width, img_height),
                                              batch_size = batch_size,
                                              class_mode='binary')

Found 17500 images belonging to 2 classes.


In [ ]:
val_generator = datagen.flow_from_directory(val_dir, 
                                              target_size=(img_width, img_height),
                                              batch_size = batch_size,
                                              class_mode='binary')

Found 3750 images belonging to 2 classes.


In [ ]:
test_generator = datagen.flow_from_directory(test_dir, 
                                              target_size=(img_width, img_height),
                                              batch_size = batch_size,
                                              class_mode='binary')

Found 3750 images belonging to 2 classes.


In [ ]:
!cp /content/train/cats/667.jpg /content/train/cats/666.jpg
!cp /content/test/dogs/11703.jpg /content/test/dogs/11702.jpg

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

In [ ]:
inception_v3_net = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(150, 150, 3))
inception_v3_net.trainable=False

In [ ]:
from tensorflow.keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Activation, Dense
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = Sequential()
model.add(inception_v3_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 256)               4718848   
                                                                 
 activation_94 (Activation)  (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
 activation_95 (Activation)  (None, 1)                 0

In [ ]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate = 1e-5)
model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train // batch_size,
                    epochs = epochs,
                    validation_data  = val_generator,
                    validation_steps = nb_val // batch_size)

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [ ]:
print(scores)

[0.10300368815660477, 0.9598599076271057]
